# Testing Prompts for generating synthetic data

## Importing libraries

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_nvidia_ai_endpoints import ChatNVIDIA

import os
import pandas as pd

# Set Pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
import os
import sys
from pathlib import Path

path = Path(os.path.dirname(os.getcwd()))
path = str(path)
print(path)
sys.path.insert(1, path)

/Users/saideepbunny/Projects/Application_Ranking_System


## Reading the data

In [3]:
df = pd.read_csv(f"{path}/data/kaggle_linkedin_jd_skills_dataset/sampled_jd_skills.csv")
df

,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len,skill_count
0,https://www.linkedin.com/jobs/view/software-ar...,2024-01-19 09:45:09.215838+00,t,t,f,Software Architect,Harris Computer,"Florida, United States",2024-01-13,Beverly Hills,United States,Director Technical,Mid senior,Onsite,"Product architecture, Engineering processes, C...",As Architect you will define the products’ hig...,340370,6407,33
1,https://www.linkedin.com/jobs/view/senior-syst...,2024-01-19 09:45:09.215838+00,t,t,f,Senior System Engineer,ExLabs,"Long Beach, CA",2024-01-13,Compton,United States,Maintenance Data Analyst,Mid senior,Onsite,"Systems engineering, Technical oversight, Requ...",Job Title: Sr. System Engineer\nCompany: ExLab...,640026,6308,32
2,https://www.linkedin.com/jobs/view/senior-data...,2024-01-20 11:15:59.804413+00,t,t,f,Senior Data Scientist,Workforce Connections,"St Louis, MO",2024-01-14,Belleville,United States,Chemist,Mid senior,Onsite,"Data Science, Data Analysis, Machine Learning,...","Location: Onsite, St. Louis MO 63146\nJob Resp...",1096469,3635,30
3,https://www.linkedin.com/jobs/view/distinguish...,2024-01-19 22:25:24.801979+00,t,t,f,Distinguished Applied Researcher,Jobs for Humanity,"Fort Worth, TX",2024-01-15,Fort Worth,United States,Director Research,Mid senior,Onsite,"AI, Applied Research, Machine Learning, Natura...",Company Description\nJobs for Humanity is part...,998553,4011,34
4,https://www.linkedin.com/jobs/view/systems-adm...,2024-01-21 01:06:09.90334+00,t,t,f,Systems Administrator,Leidos,"Colorado Springs, CO",2024-01-14,Cripple Creek,United States,Tier,Mid senior,Onsite,"System Administration, IT Operating Systems, C...",Description\nThe Leidos Digital Modernization ...,1189859,4117,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,https://uk.linkedin.com/jobs/view/workshop-con...,2024-01-19 09:45:09.215838+00,t,t,f,Workshop Controller,Activate Group Limited,"Bury St. Edmunds, England, United Kingdom",2024-01-13,Ipswich,United Kingdom,Controller,Mid senior,Onsite,"Workshop Control, Body Shop Management, HighQu...",Job title: Workshop Controller\nDepartment: Ac...,321651,4703,32
1496,https://www.linkedin.com/jobs/view/x-ray-techn...,2024-01-21 04:16:54.473867+00,t,t,f,X-Ray Technologist,Fast Pace Health,"Grant, AL",2024-01-17,Guntersville,United States,Ultrasound Technologist,Mid senior,Onsite,"XRay Technologist, HIPAA, Patient intake, Nurs...",Posting Title: X-Ray Technologist Overview:\nF...,1230454,5704,38
1497,https://www.linkedin.com/jobs/view/xfinity-ret...,2024-01-19 23:17:04.320124+00,t,t,f,Xfinity Retail Sales Consultant,XFINITY Store by Comcast,"East Brunswick, NJ",2024-01-14,Long Branch,United States,Sales-Service Promoter,Mid senior,Onsite,"Customer service, Product demonstration, Probl...",Job Summary\nResponsible for assisting and con...,159258,4839,32
1498,https://www.linkedin.com/jobs/view/yard-associ...,2024-01-19 09:45:09.215838+00,t,t,f,Yard Associate (SkillBridge),ARMOR Initiative,"Kansas City, MO",2024-01-13,Leavenworth,United States,Orderly,Associate,Onsite,"DoD SkillBridge Internship, Fulltime training,...",The ARMOR Initiative DoD SkillBridge Internshi...,774643,2277,30


## Testing prompts

In [4]:
i=1

In [5]:
print("**Qualifications:**", df.iloc[i]["job_title"])
print("**Job Role:**", df.iloc[i]["job_summary"])
print("**Experience:**", df.iloc[i]["job_skills"])


**Qualifications:** Senior System Engineer
**Job Role:** Job Title: Sr. System Engineer
Company: ExLabs
Location: Long Beach, CA
Job Type: Full-time
About ExLabs:
ExLabs is a pioneering aerospace company developing next-generation technology platforms for space exploration and resource capture. We are dedicated to pushing the boundaries of technology and innovation to unlock the vast potential of space for the benefit of humanity. We specialize in the development of advanced large-scale space vehicles and technologies, leveraging the latest innovations to tackle the challenges of space debris mitigation, object capture and control, deep space exploration, and resource acquisition.
At ExLabs our goal is to accelerate the utilization of space resources by providing unprecedented near-Earth access to natural objects. Our vehicle is being developed for object capture and trajectory alterations and redirection, providing multi-use technology capable of orbital debris mitigation, de-orbiting

### Generating Resume

In [6]:
category_requirements = {
    "complete_mismatch": """
    Resume is highly professional but entirely unsuitable for the job.

    - **Career Path:** Unrelated industry (e.g., healthcare for software engineering).  
    - **Skills:** No overlap with required skills; expertise in an unrelated domain.  
    - **Experience:** Strong accomplishments but in irrelevant fields.  
    - **Education & Certifications:** Unrelated qualifications with no relevance to the job.  
    """,
    
    "underwhelming": """
    Resume has some relevance but falls short of requirements.

    - **Experience:** 30-50% less than required; entry-level instead of mid-level.  
    - **Skills:** Lacks 40-60% of key skills; basic proficiency instead of advanced.  
    - **Projects/Work:** Limited scope, lacks complex problem-solving.  
    - **Education:** Partially relevant degree; missing specialized training.  
    """,
    
    "good": """
    Resume is a strong match for the job.

    - **Experience:** Meets role expectations; progressive career growth.  
    - **Skills:** 95-100% alignment; demonstrated real-world application.  
    - **Projects/Work:** Clear impact using Action-Problem-Result (APR) format: 'Performed [advanced strategic action] to resolve [complex systemic problem] resulting in [exceptional measurable outcomes with broad impact]'. Example: Engineered an end-to-end claim complexity scoring ML pipeline, negating ~3 minutes of manual classification per claim.
    - **Education:** Exact match with required qualifications and certifications.  
    """,
    
    "overqualified": """
    Resume significantly exceeds job requirements.

    - **Experience:** 3-5 years beyond needed; leadership roles for non-leadership positions.  
    - **Skills:** 100% match + additional high-level skills.  
    - **Projects/Work:** APR format showing strategic, high-impact contributions. APR format: 'Performed [advanced strategic action] to resolve [complex systemic problem] resulting in [exceptional measurable outcomes with broad impact]'. Example: Revised the existing claim assignment process to automatically assign claims classified by scoring engine to agents by prioritizing highly complex claims, achieving 100% automation.
    - **Education:** Advanced degrees/certifications beyond role needs.  
    - **Career Progression:** Rapid growth, senior-level achievements.  
    """
}

In [7]:
# Resume Generation Prompt for LLaMA Models

resume_template = """
You are an expert resume writer with specialized knowledge in talent acquisition and hiring practices. Your task is to generate a tailored, content-rich resume based on the provided inputs while strictly adhering to the specified fit category constraints.

## **Job Details**:
- **Role:** {job_role}
- **Job Description:** {jd}
- **Required Skills:** {skills}

## **Fit Category:** {fit_category}
## **Category Requirements:**  
{category_requirements}

## **Instructions**:
1. **Resume Objective**  
   - Generate a professional resume that precisely aligns with the specified fit category ({fit_category}).
   - Ensure the resume maintains industry standards and meets professional expectations.
   - Structure the resume to be **500-700 words** with clearly defined sections.

2. **Mandatory Sections**  
   - **Education**: University, degree title, major, graduation year, GPA.
   - **Skills**: Comma separated list of skills that include technical skills, soft skills, technologies, and other skills. List at least 15 and at most 40.
   - **Work Experience**: Consists of company names, roles, employment type, duration, and detailed bullet points. Constitutes the work experience and work done for the company. Choose a company name, employment type and duration without leaving it blank.
   

3. **Optional Sections (Context-Based)**  
   - **Projects**: Required if work experience is limited, with clear descriptions and technology stack. This constitutes of the personal and academic projects.
   - **Professional Summary**: Tailored to the fit category.
   - **Certifications/Accomplishments**: Adjusted based on qualification level.
   - **Additional Relevant Sections**: If beneficial, add elements such as leadership, research, or publications.

4. **Formatting Guidelines**  
   - Use **clear section headings**.
   - Precede each bullet point with **"-"**.
   - Mark key entities (**institutions, companies, project names**) with **"*"**.
   - Keep bullet points **≤180 characters**.
   - Include:
     - **4-6 bullet points** for work experience.
     - **2-3 bullet points** per project.

5. **Category-Specific Constraints**  
   - **Complete Mismatch (0-30% relevance)**:
     - Generate a resume from an **entirely different career path**.
     - Ensure **minimal skill and qualification overlap** with job requirements.
     - Maintain professional quality but unrelated experience.
   - **Underwhelming (50-70% relevance)**:
     - Include **some relevant experience but with clear gaps**.
     - Show a **foundational but incomplete** skill set.
     - Present **partially relevant qualifications** with notable omissions.
   - **Good Fit (95-100% relevance)**:
     - Match **all required skills and experience** at an appropriate depth.
     - Follow **Action-Problem-Result (APR) format** in work and project sections.
     - Demonstrate **clear career progression** and **precise qualification match**.
   - **Overqualified (exceeds requirements)**:
     - Include **experience exceeding job requirements by 3-5 years**.
     - Highlight **leadership, strategic impact, and advanced certifications**.
     - Showcase **complex problem-solving** and **business impact** beyond role expectations.

6. **Output Requirements**  
   - Ensure **strict adherence** to the fit category.
   - Generate **only** the resume; **do not include any explanatory notes or meta-text**.
   - Maintain **authenticity, clarity, and professionalism** throughout.
"""

In [8]:
#Llama 3.2 using Ollama on local system
model = OllamaLLM(model="llama3.2", temperature = 1)

prompt = ChatPromptTemplate.from_template(resume_template)
chain = prompt | model
data = {
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": "Good",
    "category_requirements": category_requirements["good"]
}
resume1 = chain.invoke(data)

In [9]:
print(resume1)

Here is a sample resume for a candidate with a good match for the job.

**Systems Engineer**
**Michael E. Anderson**

**Contact Information:**
- Phone: (555) 123-4567
- Email: [michaelanderson@email.com](mailto:michaelanderson@email.com)
- LinkedIn: linkedin.com/in/michael-anderson-5b6a9a12

**Summary:**
Highly motivated and experienced systems engineer with over 8 years of experience in designing, developing, testing, and maintaining complex software systems. Proven track record of delivering high-quality solutions on time and within budget. Skilled in a variety of programming languages, including C++, Java, and Python.

**Education:**

- Bachelor's Degree in Computer Science, Stanford University (2014)
- GPA: 3.8/4.0

**Skills:**
C++, Java, Python, Systems Engineering, Requirements Analysis, Design Synthesis, Technical Performance, Project Management, Subcontract Control, Risk Management, Cloud Computing, Containerization.

**Work Experience:**

- **Systems Engineer**, ABC Corporatio

In [10]:
#Llama 3.3 with NVIDIA NIM
client = ChatNVIDIA(
  model="meta/llama-3.3-70b-instruct",
  api_key="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", 
  temperature=1.0,
  top_p=0.7
)


prompt = ChatPromptTemplate.from_template(resume_template)
chain = prompt | model
data = {
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": "Good",
    "category_requirements": category_requirements["good"]
}
resume2 = chain.invoke(data)

In [11]:
print(resume2)

**Summary**

A seasoned systems engineer with a strong background in technical oversight, requirements analysis, and design synthesis. Proven track record of delivering high-quality solutions and managing complex projects from conception to completion.

**Education**

* Bachelor's Degree in Computer Science, University of California, Berkeley, CA (2015)
	+ GPA: 3.8/4.0
	+ Coursework: Data Structures, Algorithms, Computer Systems, Software Engineering

**Skills**

Systems engineering, technical oversight, requirements analysis, design synthesis, software development, project management, problem-solving, communication, teamwork, leadership.

**Technical Skills**

* Programming languages: Java, Python, C++
* Development frameworks: Spring, Django
* Databases: MySQL, PostgreSQL
* Operating Systems: Windows, Linux, macOS
* Agile methodologies: Scrum, Kanban

**Professional Summary**

Highly motivated and experienced systems engineer with a strong background in technical oversight, requireme

In [12]:
#NVIDIA nemotron model listed as a model for synthetic data generation usecase
model = ChatNVIDIA(
  model="nvidia/nemotron-4-340b-instruct",
  api_key="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", 
  temperature=1.0,
  top_p=0.7
)

prompt = ChatPromptTemplate.from_template(resume_template)
chain = prompt | model
data = {
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": "Good",
    "category_requirements": category_requirements["good"]
}
resume3 = chain.invoke(data).content

In [13]:
print(resume3)

*John Doe*

**Professional Summary**

Highly skilled Senior Systems Engineer with 12+ years of experience in designing, implementing, and maintaining complex systems. Proven track record in leading international engineering groups, providing technical oversight, and ensuring system reliability, scalability, and efficiency. Expertise in spacecraft systems engineering, requirements analysis, design synthesis, and project management.

**Skills**

Systems engineering, Technical oversight, Requirements analysis, Design synthesis, Design documentation, Technical performance, Project/program design reviews, Project/program schedule, Budget planning, Subcontract control, Risk management, Systems integration, System verification, Formal qualification, Problem-solving skills, Word, Excel, PowerPoint, MS Project, Visio, DOORs, Computer Science, Information Technology, Systems architecture, Hardware, Software, Networking, Materials, Manufacturing processes, Qualification and acceptance test standa

We can clearly see that the NVIDIA's nemotron model generated the best content. Lets check it for other category fit.

In [14]:
data = {
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": "Complete Mismatch",
    "category_requirements": category_requirements["complete_mismatch"]
}
r1 = chain.invoke(data).content
print(r1)

**Professional Summary**

Highly skilled and dedicated healthcare professional with a strong background in patient care, medical research, and team management. Proven track record of success in driving clinical operations and improving patient outcomes in fast-paced hospital environments.

**Education**

*University of California, Los Angeles (UCLA)*

- Bachelor of Science in Nursing, 2005
- GPA: 3.8

**Skills**

- Patient care, medical research, team management, clinical operations, healthcare administration, patient education, medical terminology, HIPAA compliance, electronic health records, medication administration, patient advocacy, healthcare policy, medical ethics, nursing education, patient safety, healthcare quality improvement

**Work Experience**

*Nurse Manager, Cedars-Sinai Medical Center (Los Angeles, CA)*

- Full-time, 2010-2020
	+ Oversaw daily operations of a 30-bed medical-surgical unit, managing a team of 25 nurses and support staff.
	+ Implemented evidence-based pra

In [15]:
data = {
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": "Underwhelming",
    "category_requirements": category_requirements["underwhelming"]
}
r2 = chain.invoke(data).content
print(r2)

**Professional Summary**

Recent graduate with a Bachelor's degree in Information Technology, seeking to leverage academic knowledge and limited experience in a junior-level role. Possesses basic proficiency in systems engineering and technical oversight.

**Education**

*University of California, Los Angeles*

- Bachelor of Science in Information Technology, 2022
- GPA: 3.2

**Skills**

- Systems engineering, Technical oversight, Requirements analysis, Design documentation, Project/program design reviews, Project/program schedule, Budget planning, Subcontract control, Risk management, Systems integration, System verification, Formal qualification, Problem-solving skills, Word, Excel, PowerPoint, MS Project, Visio, DOORs, Computer Science, Information Technology, Systems architecture, Hardware, Software, Networking, Materials, Manufacturing processes, Qualification and acceptance test standards, Communication, Collaboration

**Work Experience**

*IT Support Specialist, Local Tech Compa

In [16]:
data = {
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": "Underwhelming",
    "category_requirements": category_requirements["underwhelming"]
}
r2 = chain.invoke(data).content
print(r2)

**Professional Summary**

Recent graduate with a Bachelor's degree in Information Technology, seeking to leverage academic knowledge and limited experience in a junior-level role. Possesses basic proficiency in systems engineering and technical oversight.

**Education**

*University of California, Los Angeles*

- Bachelor of Science in Information Technology, 2022
- GPA: 3.2

**Skills**

- Systems engineering, Technical oversight, Requirements analysis, Design documentation, Project/program design reviews, Project/program schedule, Budget planning, Subcontract control, Risk management, Systems integration, System verification, Formal qualification, Problem-solving skills, Word, Excel, PowerPoint, MS Project, Visio, DOORs, Computer Science, Information Technology, Systems architecture, Hardware, Software, Networking, Materials, Manufacturing processes, Qualification and acceptance test standards, Communication, Collaboration

**Work Experience**

*IT Support Specialist, Local Tech Compa

In [17]:

data = {
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": "Over qualified",
    "category_requirements": category_requirements["overqualified"]
}
r3 = chain.invoke(data).content
print(r3)

**Professional Summary**

Highly accomplished and experienced Senior Systems Engineer with a proven track record of leading complex projects and driving strategic impact in the aerospace industry. Possesses a unique blend of technical expertise, leadership skills, and business acumen, with a strong focus on innovation and pushing the boundaries of space exploration.

**Education**

*University of California, Los Angeles (UCLA)*

- Master of Science in Aerospace Engineering, 2005
- Bachelor of Science in Aerospace Engineering, 2003
- GPA: 3.9/4.0

**Skills**

- Systems engineering, Technical oversight, Requirements analysis, Design synthesis, Design documentation, Technical performance, Project/program design reviews, Project/program schedule, Budget planning, Subcontract control, Risk management, Systems integration, System verification, Formal qualification, Problem-solving skills, Word, Excel, PowerPoint, MS Project, Visio, DOORs, Computer Science, Information Technology, Systems arc

NVIDIA nemotron seems to be the best model for data generation.